# Desafio Calculadora inmobiliaria

In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import statsmodels.formula.api as smf
import pysal as ps
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

E:\Miniconda3\envs\ds37\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [3]:
df = pd.read_csv("properatti.csv")
df

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,93.0,7699.115044,9354.838710,NaN,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,360.0,1383.333333,1383.333333,NaN,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Urquiza|,Argentina,Capital Federal,3433775.0,"-34.5706388726,-58.4755963355",-34.570639,...,39.0,2858.695652,3371.794872,NaN,NaN,NaN,http://www.properati.com.ar/1cja7_venta_depart...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,https://thumbs4.properati.com/9/YAe_-2gRVykADP...
121218,121218,sell,apartment,Plaza Colón,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,NaN,NaN,NaN,...,48.0,1997.916667,1997.916667,NaN,NaN,NaN,http://www.properati.com.ar/1cja8_venta_depart...,"2 Amb al contrafrente, luminoso. El departame...",2 amb. C/ dep. de servicio al contrafrente| Re...,https://thumbs4.properati.com/8/Q12PTvU6BQJ0ib...


In [4]:
df.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [6]:
place_split = df.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia',
                                                               3:'partido', 4:'localidad',
                                                               5:'barrio', 6:'zona'}, axis=1).drop(0, axis=1)

place_split.sample(15)

,pais,provincia,partido,localidad,barrio,zona
34365,Argentina,Entre Ríos,Paraná,,None,None
100861,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,,None,None
67076,Argentina,Bs.As. G.B.A. Zona Norte,Pilar,,None,None
105357,Argentina,Córdoba,Córdoba,,None,None
108545,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,La Plata,,None
54786,Argentina,Capital Federal,Liniers,,None,None
78334,Argentina,Bs.As. G.B.A. Zona Sur,Almirante Brown,Adrogué,,None
68821,Argentina,Capital Federal,,None,None,None
101622,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,Nordelta,,None
50753,Argentina,Capital Federal,Caballito,,None,None


Este código hace exactamente lo mismo y sería la forma más limpia de hacerlo
```python
df.place_with_parent_names.str.split('|', expand=True).rename({1:'pais', 2:'provincia',
                                                               3:'partido', 4:'localidad',
                                                               5:'barrio', 6:'zona'}, axis=1).drop(0, axis=1)
#Alternativa
place_split = df.place_with_parent_names.apply(lambda x: x.split('|')).apply(pd.Series)\
                            .rename({1:'pais', 2:'provincia', 3:'partido', 4:'localidad',5:'barrio', 6:'zona'}, axis=1)\
                            .drop(0, axis=1)
```

In [7]:
place_split.sample(15)

,pais,provincia,partido,localidad,barrio,zona
76799,Argentina,Capital Federal,Palermo,,None,None
34841,Argentina,Capital Federal,Balvanera,,None,None
98392,Argentina,Capital Federal,Villa Luro,,None,None
83569,Argentina,Córdoba,,None,None,None
68349,Argentina,Bs.As. G.B.A. Zona Sur,Almirante Brown,Adrogué,,None
8260,Argentina,Capital Federal,Caballito,,None,None
76694,Argentina,Bs.As. G.B.A. Zona Sur,Lomas de Zamora,Temperley,,None
101472,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,,None,None
65959,Argentina,Capital Federal,Parque Patricios,,None,None
16075,Argentina,Buenos Aires Costa Atlántica,Pinamar,,None,None


In [ ]:
pattern = []
df.head(10).description[0]

In [8]:
df.loc[df['title'].str.contains(r'amb'), 'title'].sample().to_csv()

E:\Miniconda3\envs\ds37\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


'69034,Casa de 4 ambientes con local o garaje al frente - Venta - Lanus Oeste\r\n'

In [65]:
pattern_usd = r'\b[\D$]?\$?((?!\()\d{1,3}[.,]?\d{1,3}[.,]?[09]{1,3}?)(?!\))\b'

In [20]:
pat_amientes = r'\b(\d{1,2})\s*amb'

In [28]:
df['ambientes'] = df.title.str.extract(pat_amientes, re.IGNORECASE, expand=True)

In [29]:
df.filter(['description', 'ambientes'])

,description,ambientes
0,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2
1,Venta de departamento en décimo piso al frente...,NaN
2,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2
3,PH 3 ambientes con patio. Hay 3 deptos en lote...,3
4,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,2
...,...,...
121215,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,3
121216,Excelente e impecable casa en Venta en Las Lom...,NaN
121217,VENTA DEPARTAMENTO AMBIENTE DIVISIBLE A ESTREN...,NaN
121218,"2 Amb al contrafrente, luminoso. El departame...",2


In [66]:
df['re_usd'] = df.title.str.extract(pattern_usd, re.IGNORECASE)

In [67]:
df.loc[~(df.re_usd.isnull())]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail,ambientes,re_usd
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,NaN,1400
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,NaN,1400
38,38,sell,apartment,Las Cañitas,|Argentina|Capital Federal|Las Cañitas|,Argentina,Capital Federal,NaN,"-34.5711496,-58.4232966",-34.571150,...,3692.307692,NaN,3.0,NaN,http://www.properati.com.ar/15bt2_venta_depart...,"Espectacular loft de exclusivo diseño, 210 ms,...","Venta depto,loft, Cañitas,210 ms,pileta propia...",https://thumbs4.properati.com/3/wtFrvY8S-1w-o-...,NaN,210
67,67,sell,house,Ituzaingó,|Argentina|Bs.As. G.B.A. Zona Oeste|Ituzaingó|,Argentina,Bs.As. G.B.A. Zona Oeste,3433359.0,"-34.6422298,-58.7031296",-34.642230,...,NaN,NaN,5.0,NaN,http://www.properati.com.ar/15bwz_venta_casa_i...,PLANTA ALTA: DORMITORIO EN SUITE - BALCÓNPLANT...,Chalet en Venta. Ambientes. Dormitorios. 220 m2,https://thumbs4.properati.com/6/cQAGKCZCYDfOgz...,NaN,220
84,84,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.9327656,-60.6533854",-32.932766,...,NaN,NaN,2.0,NaN,http://www.properati.com.ar/15bz0_venta_depart...,Departamento en PB con cocina comedor separado...,2 Dormitorios. Güemes al 2400,https://thumbs4.properati.com/3/p47wDv3RcA2Ppe...,NaN,2400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121110,121110,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,...,NaN,NaN,NaN,NaN,http://www.properati.com.ar/1cj3p_venta_depart...,GARCIA ANDREU -Negocios Inmobiliarios- Ofrece ...,Departamento Ricchieri al 1100 - Macrocentro,https://thumbs4.properati.com/7/DXmF-3S9_sAG2-...,NaN,1100
121117,121117,sell,apartment,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,NaN,NaN,...,1234.042553,NaN,NaN,300.0,http://www.properati.com.ar/1cj40_venta_depart...,"Departamento primer piso, Living comedor con l...",Arroyito Rio - Jose Ingenieros 1100,https://thumbs4.properati.com/2/njiXek19a_qn1D...,NaN,1100
121160,121160,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj5v_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...,NaN,1400
121209,121209,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.563685346,-58.4426825896",-34.563685,...,NaN,NaN,NaN,6300.0,http://www.properati.com.ar/1cj7c_venta_depart...,Edificio en esquina de muy buena categoría de ...,Av .Luis Maria Campos 1400- Belgrano-Capital F...,https://thumbs4.properati.com/0/zeLNBuyylOJL7z...,NaN,1400


In [64]:
df.iloc[107]['title']

'Casa 1300m² con 2 Plantas en Paz, José C. 700, San Isidro, Acassuso, por U$S 3.100.000'